<a href="https://colab.research.google.com/github/Prabhulakshman/Breast-Cancer-Prediction/blob/main/Breast_Cancer_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Breast Cancer Prediction by Building a Neural Network using PyTorch

Change the Run Time type into T4 GPU for faster performance

Import the required Dependencies

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Device Configuration

check for CUDA Availability

In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device : {device}')

Using device : cpu


Data Collection and Pre Processing

Load the Breast Cancer dataset

In [6]:
data=load_breast_cancer()
X=data.data
Y=data.target

In [7]:
X

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [8]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

Splitting Dataset into Train Data and Test Data

In [9]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [10]:
X.shape

(569, 30)

In [11]:
Y.shape

(569,)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(455, 30)
(114, 30)
(455,)
(114,)


Standardization of Data using Standary Scaler

( Ensure that the Data is Normally Distributed )

In [14]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [15]:
type(X_train)

numpy.ndarray

We cannot use numpy array type in pyTorch

Convert numpy arary into tensors and move it into GPU ( device )

In [16]:
X_train=torch.tensor(X_train,dtype=torch.float32).to(device)
X_test=torch.tensor(X_test,dtype=torch.float32).to(device)
Y_train=torch.tensor(Y_train,dtype=torch.float32).to(device)
Y_test=torch.tensor(Y_test,dtype=torch.float32).to(device)

Data Types:

[1] Scalar ( 1 )

[2] Vector ( 1010 ) -> Array

[3] Matrix ( [1 2 3 4 ])

[4] Tensor ( High dimensional Matrix -> Rubrix Cube )


Building a Neural Network Architecture

In [19]:
class NeuralNetwork(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(NeuralNetwork,self).__init__()
    # fc refers to fully connected layer
    self.fc1=nn.Linear(input_size,hidden_size) # first fully connected layer
    self.relu=nn.ReLU() # Rectified Linear Unit captures Complex Non Linear Relationships as well ( Activation Function )
    self.fc2=nn.Linear(hidden_size,output_size) # second fully connected layer
    self.sigmoid=nn.Sigmoid()
  def forward(self,x): # forward propagation x represents batch size
    out=self.fc1(x)
    out=self.relu(out)
    out=self.fc2(out)
    out=self.sigmoid(out)
    return out

Hyperparameter Tuning

In [18]:
input_size=X_train.shape[1]
hidden_size=64
output_size=1
learning_rate=0.001
num_epochs=100

Neural Network Initialization and Move it to the GPU ( de3vice )

In [20]:
model=NeuralNetwork(input_size,hidden_size,output_size).to(device)

Define Loss and Optimizer ( Binary Cross Entropy Loss )

In [21]:
criterion=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

Taining the Neural network Model

In [23]:
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad() # Resets Gradient to Zero for Each Epoch
  outputs=model(X_train) # Forwards Pass
  loss=criterion(outputs,Y_train.view(-1,1)) # Calculates Loss
  loss.backward() # Calculates Gradients ( Backward Pass)
  optimizer.step() # Updates Weights and Bias
  # Calculate Accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == Y_train.view(-1,1)).float().sum()
    accuracy = correct/Y_train.size(0)
  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

Epoch [10/100], Loss : 0.5529, Accuracy: 74.29%
Epoch [20/100], Loss : 0.4302, Accuracy: 91.65%
Epoch [30/100], Loss : 0.3406, Accuracy: 92.75%
Epoch [40/100], Loss : 0.2752, Accuracy: 93.19%
Epoch [50/100], Loss : 0.2274, Accuracy: 92.97%
Epoch [60/100], Loss : 0.1925, Accuracy: 94.07%
Epoch [70/100], Loss : 0.1667, Accuracy: 95.16%
Epoch [80/100], Loss : 0.1472, Accuracy: 95.60%
Epoch [90/100], Loss : 0.1321, Accuracy: 96.04%
Epoch [100/100], Loss : 0.1201, Accuracy: 96.26%


Model Evaluation ( Performance )

Evaluation on the Training Datasets

In [24]:
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == Y_train.view(-1,1)).float().sum()
  accuracy = correct/Y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

Accuracy on training data: 96.26%


Evaluation on the Testing Datasets

In [25]:
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == Y_test.view(-1,1)).float().sum()
  accuracy = correct/Y_test.size(0)
  print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

Accuracy on test data: 98.25%
